In [1]:
global max_bit 
max_bit = 10
max_node = 2**max_bit

In [2]:
import hashlib

In [3]:
def gethash(key):
    key = key.encode()
    h = hashlib.sha1(key)
    return int(h.hexdigest(),16)%max_node

In [4]:
key_list=[]
for i in range(max_node):
    h = gethash("%s"%i)
    key_list.append(h)

In [7]:
class ChordNode:
    def __init__(self,id):
        self.id = gethash(id)
        self.successor = None
        self.predecessor = None
        self.finger_table = {}
        self.key = []
    
    def find_successor(self,keyid):#find keyid successor
        keyid = gethash(keyid)
        if keyid == self.id: 
            #case 1:key_id = node_id
            return self
        if self == self.successor: 
            #case 2:only one node
            return self
        if self.successor.id >= keyid and self.id < keyid:
            #case 3:node_id<key_id<successor_id
            return self.successor
        if self.id > self.successor.id and keyid > self.id:
            #case 4:successor_id<node_id<key_id
            return self.successor
        if self.id > self.successor.id and keyid <= self.successor.id:
            #case 5:key_id<successor_id<self.id
            return self.successor
        else:
            self.predecessor = self.find_predecessor(keyid)
            if self.predecessor == self:
                return self.predecessor.successor.find_successor(keyid)
            else:
                return self.predecessor.find_successor(keyid)
    
    def find_predecessor(self,keyid):
        for i in range(max_bit,-1,-1):
            if self.finger_table[i].id >= self.id and self.finger_table[i].id < keyid:
                return self.finger_table[i]      
        return self

    def join(self,randNode):
        if randNode:
            self.init_finger_table(randNode)
            self.update_others()
        else:
            #only one node in the network
            for i in range(1, max_bit+1):
                self.finger_table[i] = self
            randNode.predecessor = self
    
    def init_finger_table(self,randNode):
        self.finger_table[1] = randNode.find_successor(self.finger_table[1].id)
        self.predecessor = self.successor.predecessor
        self.successor.predecessor = self
        for i in range(1,max_bit):
            if self.finger_table[i+1].id >= self.id and self.finger_table[i+1].id < self.finger_table[i].id:
                self.finger_table[i+1] = self.finger_table[i]
            else:
                self.finger_table[i+1] = randNode.find_successor(finger_table[i+1].id)
    
    def update_others(self):
        for i in range(1,m+1):
            p_id = (self.id - 2**(i-1)) % 2**max_bit
            p = self.find_successor(p_id)
            p.update_finger_table(n,i)
    
    def update_finger_table(self,s,i):
        #node s is the i_th item in finger_table of node n
        if s.id >= self.id and s.id < self.finger_table[i].id:
            self.finger_table[i] = s
            p = self.predecessor
            p.update_finger_table(s,i)
    
    def stabilize(self):
        x = self.successor.predecessor
        if x.id > self.id and x.id < self.successor.id:
            self.successor = x
        self.successor.notify(self)
        
    def notify(self,node):
        if (self.predecessor == None) or (node.id > self.predecessor.id and node.id < self.id):
            self.predecessor = node
    
    def fix_fingers(self):
        for i in range(2,max_bit+1):
            self.finger_table[i] = self.find_successor(self.finger_table[i].id)       